In [1]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
import talib as ta
from talib import MA_Type

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
#warnings.filterwarnings("ignore", category=SettingWithCopyWarning)

# Disable SettingWithCopyWarning
pd.options.mode.chained_assignment = None

In [18]:
# Define the ETF symbols
etf_symbols = ['SPY', 'QQQ', 'IWM', 'DIA']

# Define the technical indicators
sma_periods = [5, 10, 20, 50, 100, 200]
rsi_periods = [14]
macd_periods = [(12, 26)]

# Create an empty DataFrame to store the data
data_df = pd.DataFrame()

# Fetch historical price data and append to the DataFrame
for symbol in etf_symbols:
    data = yf.download(symbol, start='2021-06-29', end='2023-06-29')
    data = data[['Open', 'High', 'Low', 'Close', 'Volume']]
    data['Symbol'] = symbol
    data_df = data_df.append(data)

# Set the index to date-time
data_df.set_index(data_df.index, inplace=True)

# Calculate technical indicators
for period in sma_periods:
    sma_column_name = f"SMA {period}"
    data_df[sma_column_name] = data_df.groupby('Symbol')['Close'].transform(lambda x: ta.SMA(x.values, timeperiod=period))

for period in rsi_periods:
    rsi_column_name = f"RSI {period}"
    data_df[rsi_column_name] = data_df.groupby('Symbol')['Close'].transform(lambda x: ta.RSI(x.values, timeperiod=period))

for short_period, long_period in macd_periods:
    macd_column_name = f"MACD {short_period}-{long_period}"
    data_df[macd_column_name] = data_df.groupby('Symbol')['Close'].transform(lambda x: ta.MACD(x.values, fastperiod=short_period, slowperiod=long_period, signalperiod=9)[0])

# Calculate price change
data_df['Price Change %'] = data_df.groupby('Symbol')['Close'].transform(lambda x: x.pct_change() * 100)

# Fetch P/E Ratio for each ETF
for symbol in etf_symbols:
    ticker = yf.Ticker(symbol)
    pe_ratio = ticker.info.get('trailingPE')
    data_df.loc[data_df['Symbol'] == symbol, 'P/E Ratio'] = pe_ratio

display(data_df)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Volume,Symbol,SMA 5,SMA 10,SMA 20,SMA 50,SMA 100,SMA 200,RSI 14,MACD 12-26,Price Change %,P/E Ratio
Date,,,,,,,,,,,,,,,,
2021-06-29,427.880005,428.559998,427.130005,427.700012,35970500,SPY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.35569
2021-06-30,427.209991,428.779999,427.179993,428.059998,64827900,SPY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.084168,22.35569
2021-07-01,428.869995,430.600006,428.799988,430.429993,53441000,SPY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.553660,22.35569
2021-07-02,431.670013,434.100006,430.519989,433.720001,57697700,SPY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.764354,22.35569
2021-07-06,433.779999,434.010010,430.010010,432.929993,68710400,SPY,430.567999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.182147,22.35569
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-22,338.940002,339.929993,338.239990,339.339996,2605500,DIA,341.389996,340.923999,336.718001,336.342198,333.775199,328.596699,53.921141,1.969141,-0.026514,20.36267
2023-06-23,337.119995,338.190002,336.220001,337.160004,3720500,DIA,339.867999,340.759000,337.175500,336.353198,333.776199,328.721499,49.359272,1.632073,-0.642421,20.36267
2023-06-26,336.799988,338.079987,335.950012,337.140015,2301900,DIA,338.706000,340.546002,337.645001,336.288599,333.739299,328.825549,49.318070,1.347794,-0.005929,20.36267


In [14]:
# Define the interest rate tickers
interest_rate_tickers = ['^TNX', '^IRX', '^FVX', '^TYX']

# Create an empty DataFrame to store the data
interest_rates_df = pd.DataFrame()

# Fetch interest rate data
for ticker in interest_rate_tickers:
    interest_rates_df[ticker] = data['Close']


#interest_rates_df.reset_index(inplace=True)
interest_rates_df.head()


,^TNX,^IRX,^FVX,^TYX
Date,,,,
2021-06-29,342.859985,342.859985,342.859985,342.859985
2021-06-30,344.950012,344.950012,344.950012,344.950012
2021-07-01,346.359985,346.359985,346.359985,346.359985
2021-07-02,347.940002,347.940002,347.940002,347.940002
2021-07-06,345.820007,345.820007,345.820007,345.820007


In [6]:
# Convert index of interest_rates_df to Timestamp
interest_rates_df.index = pd.to_datetime(interest_rates_df.index)

# Merge the interest rate data with the existing DataFrame based on the date
all_data_df = pd.concat([data_df, interest_rates_df])
display(all_data_df)

,Open,High,Low,Close,Volume,Symbol,SMA 5,SMA 10,SMA 20,SMA 50,RSI 14,MACD 12-26,P/E Ratio,^TNX,^IRX,^FVX,^TYX
Date,,,,,,,,,,,,,,,,,
2022-06-29,381.230011,382.269989,378.420013,380.339996,65676000.0,SPY,NaN,NaN,NaN,NaN,NaN,NaN,22.35569,NaN,NaN,NaN,NaN
2022-06-30,376.239990,380.660004,372.559998,377.250000,112508300.0,SPY,NaN,NaN,NaN,NaN,NaN,NaN,22.35569,NaN,NaN,NaN,NaN
2022-07-01,376.559998,381.700012,373.799988,381.239990,74839700.0,SPY,NaN,NaN,NaN,NaN,NaN,NaN,22.35569,NaN,NaN,NaN,NaN
2022-07-05,375.880005,381.980011,372.899994,381.959991,81438000.0,SPY,NaN,NaN,NaN,NaN,NaN,NaN,22.35569,NaN,NaN,NaN,NaN
2022-07-06,382.109985,385.869995,379.600006,383.250000,70426200.0,SPY,380.807996,NaN,NaN,NaN,NaN,NaN,22.35569,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.799,5.135,4.046,3.872
2023-06-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.739,5.138,3.995,3.821
2023-06-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.719,5.155,3.963,3.819


In [13]:
# Group the data by ticker symbol
grouped_data = data_df.groupby('Symbol')

# Iterate over each group
for ticker_symbol, group in grouped_data:
    # Specify the filename for the CSV file
    filename = f"{ticker_symbol}_data.csv"

    # Save the group data as a CSV file
    group.to_csv(filename, index=True)